In [1]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats





from pymongo import MongoClient







from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [3]:

%time df_citation = pickle.load(open('../data/ref_dataframe_min.pkl', 'rb'))   # citation_dataframe.pkl
print ("done loading pickles", df_citation.shape)


df_citation.head()


# lista_ref_UTs = list(df_citation.reference_UT.unique() )

CPU times: user 17.6 s, sys: 3.74 s, total: 21.4 s
Wall time: 50.7 s
done loading pickles (10848620, 21)


,occurence,paper_UT,paper_char_pos,paper_word_pos,para_char_pos,para_char_total,para_index,para_word_pos,para_word_total,reference_UT,...,sect_char_pos,sect_char_total,sect_index,sect_word_pos,sect_word_total,section_title,section_title_alt,regex_sect_index,ref_datetime,plos_datetime
0,1,324515600133,139,21,139,820,0,21,130,A1995QY75100004,...,139,4029,0,21,658,Introduction,introduction,0,1995-05-15 00:00:00,2013-09-04 00:00:00
1,2,324515600133,16829,2992,494,1208,24,76,188,A1995QY75100004,...,494,5398,3,76,864,Discussion,discussion,3,1995-05-15 00:00:00,2013-09-04 00:00:00
2,3,324515600133,17257,3057,922,1208,24,141,188,A1995QY75100004,...,922,5398,3,141,864,Discussion,discussion,3,1995-05-15 00:00:00,2013-09-04 00:00:00
3,1,324515600133,266,42,266,820,0,42,130,000222499800017,...,266,4029,0,42,658,Introduction,introduction,0,2004-06-15 00:00:00,2013-09-04 00:00:00
4,2,324515600133,444,73,444,820,0,73,130,000222499800017,...,444,4029,0,73,658,Introduction,introduction,0,2004-06-15 00:00:00,2013-09-04 00:00:00


In [5]:
len(df_citation.reference_UT.unique() ) # 2,607,458

len(df_citation.paper_UT.unique() ) # 159,057


df_ref_one_per_sect = df_citation.drop_duplicates(subset=['reference_UT', 'paper_UT','sect_index'])
df_ref_one_per_sect.shape    # 7,357,998


# df_ref_one_per_plos = df_citation.drop_duplicates(subset=['reference_UT', 'paper_UT'])
# df_ref_one_per_plos.shape    #  5,772,053

159057